In [1]:
import dash

from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

host = os.getenv('HOST')
port = os.getenv('PORT')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
database= os.getenv('DATABASE')

In [3]:
connection = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
print("Verbindung zur Datenbank erfolgreich hergestellt.")

# Einen Cursor erstellen, um SQL-Abfragen auszuführen
cursor = connection.cursor()

# SQL SELECT-Abfrage für die Stationen
select_value_query = """
    SELECT v.*, b.ort
    FROM value v
    JOIN box_id b ON v.senseid_fk = b.senseid;
"""
cursor.execute(select_value_query)
values = cursor.fetchall()

df = pd.DataFrame(values, columns=[desc[0] for desc in cursor.description])

Verbindung zur Datenbank erfolgreich hergestellt.


In [4]:
stations = df['ort'].unique()
stations_options = [{'label': 'Alle Stationen', 'value': 'all'}] + [{'label': station, 'value': station} for station in stations]

In [15]:
#Nur für tests benutzten, falls Datenbank nicht verfügbar ist
df = pd.read_csv("valueDaten.csv")
df['time'] = pd.to_datetime(df['time'],format='mixed')

In [5]:
df['date'] = df['time'].dt.date
df['month'] = df["time"].dt.month

In [6]:
df.head()

,index,time,temperature,luftfeuchtigkeit,pm10,pm2_5,senseid_fk,ort,date,month
0,1422469,2022-07-29 23:09:41,17.777778,NaN,NaN,NaN,605f498077a88b001bba3dc0,Dortmund,2022-07-29,7
1,1422470,2022-07-29 23:09:11,17.777778,NaN,NaN,NaN,605f498077a88b001bba3dc0,Dortmund,2022-07-29,7
2,1422471,2022-07-29 23:08:37,17.722222,NaN,NaN,NaN,605f498077a88b001bba3dc0,Dortmund,2022-07-29,7
3,1422472,2022-07-29 23:08:06,17.722222,NaN,NaN,NaN,605f498077a88b001bba3dc0,Dortmund,2022-07-29,7
4,1422473,2022-07-29 23:07:45,17.722222,NaN,NaN,NaN,605f498077a88b001bba3dc0,Dortmund,2022-07-29,7


In [7]:
stations

array(['Dortmund', 'Mersch', 'Ettlingen', 'Hamburg', 'Stuttgart',
       'München', 'Berlin', 'Jena'], dtype=object)

In [8]:
df_agg = df[["index","date","temperature","luftfeuchtigkeit","pm2_5","pm10","month"]]

In [10]:
daily_avg_df = df_agg.groupby('date').mean().reset_index()


In [11]:
daily_avg_df.tail()

,date,index,temperature,luftfeuchtigkeit,pm2_5,pm10,month
783,2024-06-04,1.836825e+05,19.327014,66.038602,4.913342,8.124306,6.0
784,2024-06-05,5.866578e+05,19.710927,68.839773,2.512395,4.197308,6.0
785,2024-06-06,1.194072e+06,20.364297,61.053611,2.154870,4.004948,6.0
786,2024-06-07,1.278066e+06,20.771354,54.823568,3.650651,5.490712,6.0
787,2024-06-08,1.714844e+06,22.079079,55.208763,3.964697,5.691447,6.0


In [12]:
daily_avg_df.dtypes

date                 object
index               float64
temperature         float64
luftfeuchtigkeit    float64
pm2_5               float64
pm10                float64
month               float64
dtype: object

In [18]:
# Dash-App initialisieren
app = dash.Dash(__name__)

# Layout der Dash-App definieren
app.layout = html.Div(children=[
    html.H1(
        children='Umweltüberwachung Dashboard',
        style={'background-color': 'white'}
    ),

    html.Div(
        children='Wählen Sie eine Station und eine Kennzahl aus, um die Daten anzuzeigen.',
        style={'background-color': 'white'}
    ),

    dcc.Dropdown(
        id='station-dropdown',
        options=stations_options,
        value='all',
        style={'background-color': 'white'}
    ),

    html.Div(
        children='Wählen Sie eine Kennzahl:',
        style={'background-color': 'white', 'marginTop': 20}
    ),
    
    dcc.RadioItems(
        id='metric-radio',
        options=[
            {'label': 'Durchschnitt', 'value': 'mean'},
            {'label': 'Median', 'value': 'median'},
            {'label': 'Varianz', 'value': 'var'},
            {'label': 'Standardabweichung', 'value': 'std'}
        ],
        value='mean',
        labelStyle={'display': 'inline-block', 'marginRight': 10, 'background-color': 'white'}
    ),

    html.Div(
        children='Wählen Sie die Aggregationsstufe:',
        style={'background-color': 'white', 'marginTop': 20}
    ),
    
    dcc.Checklist(
        id='aggregation-checklist',
        options=[
            {'label': 'Stunde', 'value': 'hour'},
            {'label': 'Tag', 'value': 'day'},
            {'label': 'Monat', 'value': 'month'}
        ],
        value=['day'],
        labelStyle={'display': 'inline-block', 'marginRight': 10, 'background-color': 'white'}
    ),

    html.Div(
        dcc.Graph(id='temperature-graph'),
        style={'background-color': 'white'}
    ),
    html.Div(
        dcc.Graph(id='luftfeuchtigkeit-graph'),
        style={'background-color': 'white'}
    ),
    html.Div(
        dcc.Graph(id='pm-graph'),
        style={'background-color': 'white'}
    )
], style={'background-color': 'white'})

# Callback zur Aktualisierung der Diagramme
@app.callback(
    [Output('temperature-graph', 'figure'),
     Output('luftfeuchtigkeit-graph', 'figure'),
     Output('pm-graph', 'figure')],
    [Input('station-dropdown', 'value'),
     Input('metric-radio', 'value'),
     Input('aggregation-checklist', 'value')]
)
def update_graphs(selected_station, selected_metric, selected_aggregation):
    # Filtere den DataFrame nach der ausgewählten Station
    if selected_station != 'all':
        filtered_df = df[df['station_name'] == selected_station]
    else:
        filtered_df = df.copy()

    # Erstelle die Aggregationsspalte basierend auf der Auswahl
    if 'hour' in selected_aggregation and 'day' in selected_aggregation and 'month' in selected_aggregation:
        filtered_df['aggregation'] = filtered_df['time'].dt.strftime('%Y-%m-%d %H')
        aggregation_name = 'Monat:Tag:Stunde'
    elif 'day' in selected_aggregation and 'month' in selected_aggregation:
        filtered_df['aggregation'] = filtered_df['time'].dt.strftime('%Y-%m-%d')
        aggregation_name = 'Monat:Tag'
    elif 'hour' in selected_aggregation and 'day' in selected_aggregation:
        filtered_df['aggregation'] = filtered_df['time'].dt.strftime('%Y-%m-%d %H')
        aggregation_name = 'Tag:Stunde'
    elif 'month' in selected_aggregation:
        filtered_df['aggregation'] = filtered_df['time'].dt.strftime('%Y-%m')
        aggregation_name = 'Monat'
    elif 'day' in selected_aggregation:
        filtered_df['aggregation'] = filtered_df['time'].dt.dayofyear
        aggregation_name = 'Tag'
    elif 'hour' in selected_aggregation:
        filtered_df['aggregation'] = filtered_df['time'].dt.hour
        aggregation_name = 'Stunde'
    else:
        filtered_df['aggregation'] = filtered_df['time'].dt.strftime('%Y-%m')
        aggregation_name = 'Monat'

    # Bereinige den DataFrame von NaN-Werten
    filtered_df = filtered_df.dropna(subset=['temperature', 'luftfeuchtigkeit', 'pm2_5', 'pm10'])

    # Berechne die Kennzahlen
    daily_df = filtered_df.groupby('aggregation').agg({
        'temperature': selected_metric,
        'luftfeuchtigkeit': selected_metric,
        'pm2_5': selected_metric,
        'pm10': selected_metric
    }).reset_index()

    # Erstelle Diagramme
    temp_fig = px.line(daily_df, x='aggregation', y='temperature', title=f'{selected_metric.capitalize()} Temperatur', labels={'aggregation': aggregation_name})
    humidity_fig = px.line(daily_df, x='aggregation', y='luftfeuchtigkeit', title=f'{selected_metric.capitalize()} Luftfeuchtigkeit', labels={'aggregation': aggregation_name})
    pm_fig = px.line(daily_df, x='aggregation', y=['pm2_5', 'pm10'], title=f'{selected_metric.capitalize()} PM2.5 und PM10', labels={'aggregation': aggregation_name})

    return temp_fig, humidity_fig, pm_fig

# App ausführen
if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8050)